In [1]:
import mindspore
import time
from mindspore import nn
from mindspore import ops
from mindspore.dataset import vision, transforms
from mindspore.dataset import Cifar10Dataset

In [2]:
import mindspore.context as context
context.set_context(device_target="GPU")

In [3]:
# Download data from open datasets
'''
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/cifar10.zip"
path = download(url, "./", kind="zip", replace=True)
'''

'\nfrom download import download\n\nurl = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/"       "notebook/datasets/cifar10.zip"\npath = download(url, "./", kind="zip", replace=True)\n'

In [4]:
train_dataset = Cifar10Dataset('cifar10/train')
test_dataset = Cifar10Dataset('cifar10/test')

In [5]:
print(train_dataset.get_col_names())

['image', 'label']


In [6]:
def datapipe(dataset, batch_size):
    image_transforms = [
        vision.Rescale(1.0 / 255.0, 0),
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        vision.HWC2CHW()
    ]
    label_transform = transforms.TypeCast(mindspore.int32)

    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    dataset = dataset.batch(batch_size)
    return dataset

In [7]:
# Map vision transforms and batch dataset
train_dataset = datapipe(train_dataset, 64)
test_dataset = datapipe(test_dataset, 64)

In [8]:
for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break

Shape of image [N, C, H, W]: (64, 3, 32, 32) Float32
Shape of label: (64,) Int32


In [9]:
for data in test_dataset.create_dict_iterator():
    print(f"Shape of image [N, C, H, W]: {data['image'].shape} {data['image'].dtype}")
    print(f"Shape of label: {data['label'].shape} {data['label'].dtype}")
    break

Shape of image [N, C, H, W]: (64, 3, 32, 32) Float32
Shape of label: (64,) Int32


In [10]:
# Define model
class Network(nn.Cell):
    def __init__(self, num_class=10, num_channel=3):
        super().__init__()
        self.sequential = nn.SequentialCell(
            nn.Conv2d(num_channel, 64, 3, pad_mode='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=64),
            nn.Conv2d(64, 64, 3, pad_mode='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=64),
            
            nn.Conv2d(64, 128, 3, pad_mode='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=128),
            nn.Conv2d(128, 128, 3, pad_mode='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, 3, pad_mode='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=256),
            nn.Conv2d(256, 256, 3, pad_mode='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=256),
            nn.Conv2d(256, 256, 3, pad_mode='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=256),
            
            nn.Conv2d(256, 512, 3, pad_mode='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=512),
            nn.Conv2d(512, 512, 3, pad_mode='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=512),
            nn.Conv2d(512, 512, 3, pad_mode='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=512),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(512, 512, 3, pad_mode='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=512),
            nn.Conv2d(512, 512, 3, pad_mode='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=512),
            nn.Conv2d(512, 512, 3, pad_mode='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=512),
            
            nn.Flatten(),
            nn.Dense(8*8*512, 4096),
            nn.ReLU(),
            nn.Dense(4096, 4096),
            nn.ReLU(),
            nn.Dense(4096, 1000),
            nn.Dropout(keep_prob=0.9),
            nn.Dense(1000, 10)
        )

    def construct(self, x):
        logits = self.sequential(x)
        return logits

model = Network(num_class=10)

In [11]:
# Instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
# optimizer = nn.SGD(model.trainable_params(), 1e-2)
optimizer = nn.Momentum(model.trainable_params(), 1e-2, momentum=0.85)

In [12]:
def train(model, dataset, loss_fn, optimizer):
    # Define forward function
    def forward_fn(data, label):
        logits = model(data)
        loss = loss_fn(logits, label)
        return loss, logits

    # Get gradient function
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    # Define function of one-step training
    def train_step(data, label):
        (loss, _), grads = grad_fn(data, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    size = dataset.get_dataset_size()
    model.set_train()
    sumloss = 0
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)
        sumloss = sumloss + loss.asnumpy()
        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(sumloss)
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")
        if batch == 781:
            print("avgloss:{}".format(sumloss/782))

In [13]:
def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset, loss_fn, optimizer)
    test(model, test_dataset, loss_fn)
print("Done!")

Epoch 1
-------------------------------
2.3229317665100098
loss: 2.322932  [  0/782]
191.25473988056183
loss: 1.586818  [100/782]
347.59121000766754
loss: 1.616058  [200/782]
494.2684851884842
loss: 1.422984  [300/782]
625.4775899648666
loss: 1.494088  [400/782]
751.1060481071472
loss: 1.030253  [500/782]
866.6730900406837
loss: 0.921062  [600/782]
977.1015055775642
loss: 0.837721  [700/782]
avgloss:1.355538042036205
Test: 
 Accuracy: 63.9%, Avg loss: 1.029226 

Epoch 2
-------------------------------
0.6807944774627686
loss: 0.680794  [  0/782]
91.86119985580444
loss: 0.958165  [100/782]
178.889790892601
loss: 1.064821  [200/782]


In [ ]:
# Save checkpoint
# mindspore.save_checkpoint(model, "model.ckpt")
# print("Saved Model to model.ckpt")